In [1]:
import scrapy
from scrapy.crawler import CrawlerProcess
import re

In [2]:
radiolab_data = []

In [3]:
# only season 3 has legitimate links
# parses transcript links
class radiolab(scrapy.Spider):
    name = 'radiolab'
    
    def start_requests(self):
        base_url = 'https://www.wnycstudios.org/podcasts/radiolab/podcasts/'
        for i in range(1, 30):
            url = base_url + str(i)
            yield scrapy.Request(url=url, callback=self.parse_episode)
        
    def parse_episode(self, response):
        episode_links = response.xpath('//h1[contains(@class,episode-tease__title)]//a/@href').extract()
        for link in episode_links:
            link = 'https://www.wnycstudios.org' + link
            # print(link)
            yield response.follow(url=link, 
                               callback =self.parse_transcript) 
    

    def parse_transcript(self, response):
        current_ep = []
        
        title = response.xpath('//h3[@class="story__title"]/text()').extract()
        current_ep.append(title)
        
        year = response.xpath('//p[@class="story-metadata__date"]/text()').extract()
        current_ep.append(year)
        
        text = response.xpath('//div[@class="text"]//text()').extract()
        current_ep.append(text)
        
        radiolab_data.append(current_ep)

In [4]:
process = CrawlerProcess()
process.crawl(radiolab)
process.start()

2021-03-22 22:00:13 [scrapy.utils.log] INFO: Scrapy 2.4.1 started (bot: scrapybot)
2021-03-22 22:00:13 [scrapy.utils.log] INFO: Versions: lxml 4.6.1.0, libxml2 2.9.10, cssselect 1.1.0, parsel 1.6.0, w3lib 1.22.0, Twisted 20.3.0, Python 3.8.5 (default, Sep  3 2020, 21:29:08) [MSC v.1916 64 bit (AMD64)], pyOpenSSL 19.1.0 (OpenSSL 1.1.1h  22 Sep 2020), cryptography 3.1.1, Platform Windows-10-10.0.19041-SP0
2021-03-22 22:00:13 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.selectreactor.SelectReactor
2021-03-22 22:00:13 [scrapy.crawler] INFO: Overridden settings:
{}
2021-03-22 22:00:13 [scrapy.extensions.telnet] INFO: Telnet Password: 27bc757017703d0b
2021-03-22 22:00:13 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.logstats.LogStats']
2021-03-22 22:00:13 [scrapy.middleware] INFO: Enabled downloader middlewares:
['scrapy.downloadermiddlewares.httpauth.HttpAuthMiddleware',
 's

2021-03-22 22:00:18 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.wnycstudios.org/podcasts/radiolab/episodes/60-words> (referer: https://www.wnycstudios.org/podcasts/radiolab/podcasts/7)
2021-03-22 22:00:18 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.wnycstudios.org/podcasts/radiolab/articles/body-count> (referer: https://www.wnycstudios.org/podcasts/radiolab/podcasts/7)
2021-03-22 22:00:18 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.wnycstudios.org/podcasts/radiolab/podcasts/8> (referer: None)
2021-03-22 22:00:18 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.wnycstudios.org/podcasts/radiolab/articles/bobbys> (referer: https://www.wnycstudios.org/podcasts/radiolab/podcasts/7)
2021-03-22 22:00:18 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.wnycstudios.org/podcasts/radiolab/articles/other-latif-episode-2> (referer: https://www.wnycstudios.org/podcasts/radiolab/podcasts/7)
2021-03-22 22:00:18 [scrapy.core.engine] DEBUG

2021-03-22 22:00:20 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.wnycstudios.org/podcasts/radiolab/articles/debatable> (referer: https://www.wnycstudios.org/podcasts/radiolab/podcasts/21)
2021-03-22 22:00:21 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.wnycstudios.org/podcasts/radiolab/articles/i-dont-have-answer> (referer: https://www.wnycstudios.org/podcasts/radiolab/podcasts/21)
2021-03-22 22:00:21 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.wnycstudios.org/podcasts/radiolab/podcasts/16> (referer: None)
2021-03-22 22:00:21 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.wnycstudios.org/podcasts/radiolab/articles/grays-donation> (referer: https://www.wnycstudios.org/podcasts/radiolab/podcasts/22)
2021-03-22 22:00:23 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.wnycstudios.org/podcasts/radiolab/podcasts/17> (referer: None)
2021-03-22 22:00:23 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.wnycstudios.org/p

2021-03-22 22:00:28 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.wnycstudios.org/podcasts/radiolab/articles/invisible-girl> (referer: https://www.wnycstudios.org/podcasts/radiolab/podcasts/19)
2021-03-22 22:00:28 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.wnycstudios.org/podcasts/radiolab/articles/update-eye-sky> (referer: https://www.wnycstudios.org/podcasts/radiolab/podcasts/19)
2021-03-22 22:00:28 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.wnycstudios.org/podcasts/radiolab/articles/primitive-streak> (referer: https://www.wnycstudios.org/podcasts/radiolab/podcasts/19)
2021-03-22 22:00:29 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.wnycstudios.org/podcasts/radiolab/articles/edge> (referer: https://www.wnycstudios.org/podcasts/radiolab/podcasts/20)
2021-03-22 22:00:29 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.wnycstudios.org/podcasts/radiolab/articles/alpha-gal> (referer: https://www.wnycstudios.org/podcasts/

2021-03-22 22:00:32 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.wnycstudios.org/podcasts/radiolab/articles/fronads> (referer: https://www.wnycstudios.org/podcasts/radiolab/podcasts/14)
2021-03-22 22:00:32 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.wnycstudios.org/podcasts/radiolab/articles/theprimordialjourney> (referer: https://www.wnycstudios.org/podcasts/radiolab/podcasts/14)
2021-03-22 22:00:32 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.wnycstudios.org/podcasts/radiolab/articles/birthstory2018> (referer: https://www.wnycstudios.org/podcasts/radiolab/podcasts/14)
2021-03-22 22:00:33 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.wnycstudios.org/podcasts/radiolab/articles/fu-go> (referer: https://www.wnycstudios.org/podcasts/radiolab/podcasts/10)
2021-03-22 22:00:33 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.wnycstudios.org/podcasts/radiolab/articles/dinopocalypse-redux> (referer: https://www.wnycstudios.org/p

2021-03-22 22:00:35 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.wnycstudios.org/podcasts/radiolab/episodes/translation> (referer: https://www.wnycstudios.org/podcasts/radiolab/podcasts/4)
2021-03-22 22:00:35 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.wnycstudios.org/podcasts/radiolab/articles/radiolab-enemy-mankind> (referer: https://www.wnycstudios.org/podcasts/radiolab/podcasts/2)
2021-03-22 22:00:37 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.wnycstudios.org/podcasts/radiolab/articles/border-trilogy-part-3-what-remains> (referer: https://www.wnycstudios.org/podcasts/radiolab/podcasts/14)
2021-03-22 22:00:37 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.wnycstudios.org/podcasts/radiolab/articles/note-radiolab-team> (referer: https://www.wnycstudios.org/podcasts/radiolab/podcasts/2)
2021-03-22 22:00:37 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.wnycstudios.org/podcasts/radiolab/articles/bringing-gamma-back> (re

2021-03-22 22:00:44 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.wnycstudios.org/podcasts/radiolab/articles/276577-rebroadcast-emergence> (referer: https://www.wnycstudios.org/podcasts/radiolab/podcasts/28)
2021-03-22 22:00:44 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.wnycstudios.org/podcasts/radiolab/episodes/317421-blame> (referer: https://www.wnycstudios.org/podcasts/radiolab/podcasts/27)
2021-03-22 22:00:44 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.wnycstudios.org/podcasts/radiolab/articles/poop-train> (referer: https://www.wnycstudios.org/podcasts/radiolab/podcasts/27)
2021-03-22 22:00:44 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.wnycstudios.org/podcasts/radiolab/articles/kg> (referer: https://www.wnycstudios.org/podcasts/radiolab/podcasts/25)
2021-03-22 22:00:45 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.wnycstudios.org/podcasts/radiolab/articles/303638-curious-sounds-solid-sound-festival> (referer: 

2021-03-22 22:00:50 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.wnycstudios.org/podcasts/radiolab/episodes/patient-zero-updated> (referer: https://www.wnycstudios.org/podcasts/radiolab/podcasts/24)
2021-03-22 22:00:50 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.wnycstudios.org/podcasts/radiolab/articles/outside-westgate> (referer: https://www.wnycstudios.org/podcasts/radiolab/podcasts/24)
2021-03-22 22:00:50 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.wnycstudios.org/podcasts/radiolab/articles/invisibilia> (referer: https://www.wnycstudios.org/podcasts/radiolab/podcasts/24)
2021-03-22 22:00:51 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.wnycstudios.org/podcasts/radiolab/articles/262588-bitter-end> (referer: https://www.wnycstudios.org/podcasts/radiolab/podcasts/29)
2021-03-22 22:00:51 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.wnycstudios.org/podcasts/radiolab/articles/juicervose> (referer: https://www.wnycstud

In [5]:
radiolab_data[2]

[['\n    Sight Unseen\n  '],
 ['\n    January 13, 2021\n  '],
 ['\n          ',
  'UNIDENTIFIED PERSON #1: Listener-supported WNYC Studios.',
  '\r\n',
  '(SOUNDBITE OF RADIOLAB INTRO)',
  '\r\n',
  "UNIDENTIFIED PERSON #2: Wait. Wait. You're listening (laughter)...",
  '\r\n',
  'UNIDENTIFIED PERSON #3: OK.',
  '\r\n',
  'UNIDENTIFIED PERSON #2: All right.',
  '\r\n',
  'UNIDENTIFIED PERSON #3: OK.',
  '\r\n',
  'UNIDENTIFIED PERSON #2: All right.',
  '\r\n',
  "UNIDENTIFIED PERSON #3: You're listening...",
  '\r\n',
  'UNIDENTIFIED PERSON #2: Listening...',
  '\r\n',
  'UNIDENTIFIED PERSON #3: ...To RADIOLAB.',
  '\r\n',
  'UNIDENTIFIED PERSON #4: RADIOLAB.',
  '\r\n',
  'UNIDENTIFIED PEOPLE: From WNYC.',
  '\r\n',
  'UNIDENTIFIED PERSON #3: C?',
  '\r\n',
  'UNIDENTIFIED PERSON #2: Yep.',
  '\r\n',
  "LULU MILLER: I'm Lulu Miller. This is RADIOLAB.",
  '\r\n',
  '(SOUNDBITE OF TYPING)',
  '\r\n',
  "LULU MILLER: And today we're going to start with...",
  '\r\n',
  'KAINAZ AMARIA: Lu

In [6]:
import pandas as pd

radiolab_df = pd.DataFrame(radiolab_data,
                         columns = ['Title','Year','Text'])
radiolab_df

,Title,Year,Text
0,[\n Fungus Amungus\n ],"[\n September 4, 2020\n ]","[\n , (SOUNDBITE OF MUSIC), \r\n, UNI..."
1,[\n How to Be a Hero\n ],"[\n January 9, 2018\n ]","[\n , , \r\n, [RADIOLAB INTRO], \r\n..."
2,[\n Sight Unseen\n ],"[\n January 13, 2021\n ]","[\n , UNIDENTIFIED PERSON #1: Listene..."
3,[\n More Money Less Problems\n ],"[\n January 15, 2021\n ]","[\n , ANNOUNCER: Listener-supported W..."
4,[\n Post Reports: Four Hours of Insurrectio...,"[\n January 16, 2021\n ]","[\n , Speaker: , Wait, you're listeni..."
...,...,...,...
256,[\n Eye in the Sky\n ],"[\n June 18, 2015\n ]","[\n , Title Sequence:, \r\n, Wait, wa..."
257,[\n Worth\n ],"[\n December 23, 2014\n ]","[\n , WORTH FINAL WEB TRANSCRIPT, \r\..."
258,[\n Mau Mau \n ],"[\n July 3, 2015\n ]","[\n , Speaker 3:, \r\n, Wait, wait, y..."
259,[\n American Football\n ],"[\n January 29, 2015\n ]","[\n , AMERICAN FOOTBALL FINAL WEB TRA..."


In [7]:
# clean title
radiolab_df['Title'] = radiolab_df.Title.map(lambda t: t[0].split('\n')[1])
radiolab_df.Title[:5]

0                                  Fungus Amungus
1                                How to Be a Hero
2                                    Sight Unseen
3                        More Money Less Problems
4        Post Reports: Four Hours of Insurrection
Name: Title, dtype: object

In [8]:
# clean year
radiolab_df['Year'] = radiolab_df.Year.map(lambda d: d[0].split('\n')[1][-4:])
radiolab_df.Year[-10:]

251    2015
252    2015
253    2015
254    2015
255    2015
256    2015
257    2014
258    2015
259    2015
260    2015
Name: Year, dtype: object

In [9]:
# clean text
radiolab_df.Text[2]

['\n          ',
 'UNIDENTIFIED PERSON #1: Listener-supported WNYC Studios.',
 '\r\n',
 '(SOUNDBITE OF RADIOLAB INTRO)',
 '\r\n',
 "UNIDENTIFIED PERSON #2: Wait. Wait. You're listening (laughter)...",
 '\r\n',
 'UNIDENTIFIED PERSON #3: OK.',
 '\r\n',
 'UNIDENTIFIED PERSON #2: All right.',
 '\r\n',
 'UNIDENTIFIED PERSON #3: OK.',
 '\r\n',
 'UNIDENTIFIED PERSON #2: All right.',
 '\r\n',
 "UNIDENTIFIED PERSON #3: You're listening...",
 '\r\n',
 'UNIDENTIFIED PERSON #2: Listening...',
 '\r\n',
 'UNIDENTIFIED PERSON #3: ...To RADIOLAB.',
 '\r\n',
 'UNIDENTIFIED PERSON #4: RADIOLAB.',
 '\r\n',
 'UNIDENTIFIED PEOPLE: From WNYC.',
 '\r\n',
 'UNIDENTIFIED PERSON #3: C?',
 '\r\n',
 'UNIDENTIFIED PERSON #2: Yep.',
 '\r\n',
 "LULU MILLER: I'm Lulu Miller. This is RADIOLAB.",
 '\r\n',
 '(SOUNDBITE OF TYPING)',
 '\r\n',
 "LULU MILLER: And today we're going to start with...",
 '\r\n',
 'KAINAZ AMARIA: Lulu?',
 '\r\n',
 "LULU MILLER: I'm just recording your furious typing.",
 '\r\n',
 '(LAUGHTER)',
 '

In [10]:
# clean text
import re

def nofiller(text):
    text = ' '.join(text)
    text = text.replace('\r', '').replace('\n', '')
    text = text.replace(u'\xa0',u'')
    return text

nofiller(radiolab_df.Text[10])

'             [RADIOLAB INTRO]    LULU MILLER:  Picture the person you love the most. Picture them sitting on the couch, eating cereal, ranting about something totally charming. Like how it bothers them when people sign their emails with a single initial instead of just taking those four extra keystrokes to finish the job. Chaos will get them. Chaos will crack them from the outside with a falling branch, a speeding car, a bullet. Or unravel them from the inside with the mutiny of their very own cells. Chaos will rot your plants and kill your dog and rust your bike. It will decay your most precious memories, topple your favorite cities, wreck any sanctuary you can ever build. It’s not if, it’s when. Chaos is the only sure thing in this world. The master that rules us all. My scientist father taught me early that there is no escaping the second law of thermodynamics. Entropy is only growing, it can never be diminished, no matter what we do. A smart human accepts this truth. A smart human

In [11]:
radiolab_df['Text'] = radiolab_df.Text.map(lambda t: nofiller(t))
radiolab_df.Text[12]

'             [RADIOLAB INTRO]    JAD ABUMRAD:  Hey, I\'m Jad. This is Radiolab. I want to play a story that\'s come up a couple of times in conversations with folks here at the show, and I guess I\'ll preface it this way. So one of the things that\'s been a little spooky weird about this moment is just time. You know, at least for those of us who don\'t have to work on the frontlines and are lucky enough to still have jobs, we are stuck in our homes doing the Zooms, trying to get things done, but the -- just the lack of routines, routines that typically give a day purpose, without those routines time does weird things. It bleeds, it stretches and then collapses. I mean, we\'ve all had the experience of talking about something that happened on Tuesday, and then suddenly we\'re like, "Oh my God, that was just Tuesday? That feels like a lifetime ago." On the flip side, there\'s April. The entire month of April which lasted a second.    JAD:  And I found myself thinking back to one of the

In [12]:
%store radiolab_df

Stored 'radiolab_df' (DataFrame)
